### INTENT AND EMOTION DETECTION USING DistilBERT

In [1]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [2]:
# Define intent keywords
intent_keywords = {
    "venting": ["frustrated", "angry", "fed up", "tired", "exhausted"],
    "seeking_advice": ["what should I do", "any suggestions", "advice", "recommend"],
    "gratitude": ["thank you", "appreciate", "grateful", "thanks a lot"],
    "general_chat": ["how are you", "what's up", "hello", "hi"],
}
# Define personas
persona_styles = {
    "companion": lambda msg: f"I'm right here with you. {msg} We can face this together.",
    "coach": lambda msg: f"Alright, let's break it down. {msg} You’ve got what it takes.",
    "listener": lambda msg: f"I'm listening... {msg} Take your time, I'm here for it.",
}

In [3]:
# Get user input
user_input = input("Enter a sentence: ")
chosen_persona = input("Choose a persona (companion, coach, listener): ")

Enter a sentence: I have been having a bad day.
Choose a persona (companion, coach, listener): companion


In [4]:
# Intent detection
def detect_intent(text):
  text = text.lower()
  for intent, keywords in intent_keywords.items():
    if any(keyword in text for keyword in keywords):
      return intent
  return "general_chat"
intent = detect_intent(user_input)

In [5]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nateraw/bert-base-uncased-emotion')
model = AutoModelForSequenceClassification.from_pretrained('nateraw/bert-base-uncased-emotion')

# Preprocess input
inputs = tokenizer(user_input, return_tensors="pt")

# Run through the model
with torch.no_grad():
  outputs = model(**inputs)

# Get emotion probabilities
probs = F.softmax(outputs.logits, dim=1)
predicted_emotion = torch.argmax(probs, dim=1).item()
emotion_label = model.config.id2label[predicted_emotion]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:
# Respond based on emotion
base_response = {
    "sadness": "I'm really sorry you're feeling this way. Want to talk more about what's weighing you down?",
    "joy": "That's great to hear! Tell me more about what made you feel this way.",
    "love": "That sounds really heartwarming. Want to share more?",
    "anger": "That sounds frustrating. I'm here if you need to vent.",
    "fear": "It’s okay to feel scared sometimes. Would it help to break it down together?",
    "surprise": "Oh wow! That must have been unexpected. Tell me what happened.",
}.get(emotion_label, "I'm here for you. Want to tell me more?")

TEXT GENERATION

In [7]:
# Import libraries
from transformers import AutoModelForCausalLM

In [22]:
# Load the generator
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-0106")
model = AutoModelForCausalLM.from_pretrained("openchat/openchat-3.5-0106")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [16]:
# Simulated results from previous modules
emotion = emotion_label
persona = chosen_persona

# Build custom prompt
prompt = f"I'm feeling overwhelmed today."

In [23]:
# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(
    **inputs,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_k=50,
    top_p=0.95,
)

# Decode the response
response = tokenizer.decode(output[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

# Extract only bot's response
print(response.strip())

I have a lot of things to do and not enough time to do them. I have a deadline to meet for an important project, and I'm feeling the pressure. I know that I need to stay focused and work hard to get it done, but I'm also feeling stressed and anxious.

I know that I can't let this overwhelm me, but it's hard not to let it get to me. I need to find a way to calm down and focus
